In [1]:
# https://github.com/Thehunk1206/Zero-DCE

In [45]:
import cv2
import numpy as np
import os
from pathlib import Path
from statistics import mean, stdev
from preprocessing.single_image_enhance_tflite import zeroDCE

def find_bright_pixels(image_path, brightness_threshold=150, roi=None):
    """
    이미지에서 특정 밝기 이상인 픽셀들의 위치와 값을 추출

    Parameters:
    image_path: 이미지 파일 경로
    brightness_threshold: 밝기 임계값 (0-255)
    roi: Region of Interest (x, y, width, height) - 관심 영역 지정
    """
    # 이미지 읽기
    img = cv2.imread(image_path)

    # 그레이스케일로 변환
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # ROI 설정
    if roi is not None:
        x, y, w, h = roi
        gray = gray[y:y+h, x:x+w]
        img = img[y:y+h, x:x+w]

    # 밝기가 임계값 이상인 픽셀의 좌표 찾기
    bright_points = np.where(gray >= brightness_threshold)
    coordinates = list(zip(bright_points[1], bright_points[0]))  # (x, y) 형태로 변환

    # 결과 저장할 리스트
    results = []

    # 각 좌표에 대한 정보 수집
    for x, y in coordinates:
        pixel_value = img[y, x]  # BGR 값
        if roi is not None:
            actual_x = x + roi[0]  # 전체 이미지에서의 실제 x 좌표
            actual_y = y + roi[1]  # 전체 이미지에서의 실제 y 좌표
        else:
            actual_x = x
            actual_y = y

        results.append({
            'position': (actual_x, actual_y),
            'BGR_value': pixel_value,
            'brightness': gray[y, x]
        })

    return results

def calculate_height(pixels):
    """픽셀들의 높이 계산"""
    if not pixels:
        return 0
    y_coordinates = [pixel['position'][1] for pixel in pixels]
    return max(y_coordinates) - min(y_coordinates)

def analyze_pixel_positions(results):
    """
    픽셀 위치의 일관성을 분석하는 함수
    """
    positions = {
        'roi1': [],
        'roi2': []
    }

    # 각 이미지에서 픽셀 위치 수집
    for image_file, data in results.items():
        # ROI1 픽셀 중심점 계산
        if data['roi1']['pixels']:
            x_coords = [p['position'][0] for p in data['roi1']['pixels']]
            y_coords = [p['position'][1] for p in data['roi1']['pixels']]
            center_x = sum(x_coords) / len(x_coords)
            center_y = sum(y_coords) / len(y_coords)
            positions['roi1'].append((center_x, center_y))

        # ROI2 픽셀 중심점 계산
        if data['roi2']['pixels']:
            x_coords = [p['position'][0] for p in data['roi2']['pixels']]
            y_coords = [p['position'][1] for p in data['roi2']['pixels']]
            center_x = sum(x_coords) / len(x_coords)
            center_y = sum(y_coords) / len(y_coords)
            positions['roi2'].append((center_x, center_y))

    # 통계 계산
    stats = {}

    if positions['roi1']:
        roi1_x = [p[0] for p in positions['roi1']]
        roi1_y = [p[1] for p in positions['roi1']]
        stats['roi1_x_center_mean'] = mean(roi1_x)
        stats['roi1_x_center_std'] = stdev(roi1_x) if len(roi1_x) > 1 else 0
        stats['roi1_y_center_mean'] = mean(roi1_y)
        stats['roi1_y_center_std'] = stdev(roi1_y) if len(roi1_y) > 1 else 0

    if positions['roi2']:
        roi2_x = [p[0] for p in positions['roi2']]
        roi2_y = [p[1] for p in positions['roi2']]
        stats['roi2_x_center_mean'] = mean(roi2_x)
        stats['roi2_x_center_std'] = stdev(roi2_x) if len(roi2_x) > 1 else 0
        stats['roi2_y_center_mean'] = mean(roi2_y)
        stats['roi2_y_center_std'] = stdev(roi2_y) if len(roi2_y) > 1 else 0

    return positions, stats

# def calculate_inner_border_height(pixels, min_separation=5):
#     """ROI2의 위쪽과 아래쪽 픽셀 라인의 gap 계산 (아래쪽 라인의 최상단 기준)"""
#     if not pixels:
#         return 0, [], [], []

#     # x 좌표별로 픽셀들을 그룹화
#     x_grouped_pixels = {}
#     for pixel in pixels:
#         x = pixel['position'][0]
#         y = pixel['position'][1]
#         if x not in x_grouped_pixels:
#             x_grouped_pixels[x] = []
#         x_grouped_pixels[x].append(y)

#     gaps = []
#     top_points = []
#     bottom_points = []

#     # 각 x 좌표에서 위쪽과 아래쪽 픽셀 찾기
#     for x in sorted(x_grouped_pixels.keys()):
#         y_coords = sorted(x_grouped_pixels[x])

#         # y 좌표들을 클러스터링
#         clusters = []
#         current_cluster = [y_coords[0]]

#         for y in y_coords[1:]:
#             if y - current_cluster[-1] > min_separation:
#                 clusters.append(current_cluster)
#                 current_cluster = [y]
#             else:
#                 current_cluster.append(y)
#         clusters.append(current_cluster)

#         # 최소 2개의 클러스터가 있는 경우만 처리
#         if len(clusters) >= 2:
#             top = min(clusters[0])      # 위쪽 라인의 y 좌표
#             bottom = min(clusters[-1])   # 아래쪽 라인의 최상단 y 좌표 (변경된 부분)
#             gap = bottom - top
#             gaps.append(gap)
#             top_points.append((x, top))
#             bottom_points.append((x, bottom))

#     if gaps:
#         avg_gap = sum(gaps) / len(gaps)
#         return avg_gap, top_points, bottom_points, gaps

#     return 0, [], [], []

def calculate_inner_border_height(pixels, min_separation=5):
    """ROI2의 위쪽 라인 최하단과 아래쪽 라인 최상단 사이의 gap 계산"""
    if not pixels:
        return 0, [], [], []

    # x 좌표별로 픽셀들을 그룹화
    x_grouped_pixels = {}
    for pixel in pixels:
        x = pixel['position'][0]
        y = pixel['position'][1]
        if x not in x_grouped_pixels:
            x_grouped_pixels[x] = []
        x_grouped_pixels[x].append(y)

    gaps = []
    top_points = []
    bottom_points = []

    # 각 x 좌표에서 위쪽과 아래쪽 픽셀 찾기
    for x in sorted(x_grouped_pixels.keys()):
        y_coords = sorted(x_grouped_pixels[x])

        # y 좌표들을 클러스터링
        clusters = []
        current_cluster = [y_coords[0]]

        for y in y_coords[1:]:
            if y - current_cluster[-1] > min_separation:
                clusters.append(current_cluster)
                current_cluster = [y]
            else:
                current_cluster.append(y)
        clusters.append(current_cluster)

        # 최소 2개의 클러스터가 있는 경우만 처리
        if len(clusters) >= 2:
            top_max = max(clusters[0])      # 위쪽 라인의 최하단 y 좌표
            bottom_min = min(clusters[-1])   # 아래쪽 라인의 최상단 y 좌표
            gap = bottom_min - top_max      # 최하단과 최상단 사이의 gap
            gaps.append(gap)
            top_points.append((x, top_max))    # 위쪽 라인의 최하단 점
            bottom_points.append((x, bottom_min))  # 아래쪽 라인의 최상단 점

    if gaps:
        avg_gap = sum(gaps) / len(gaps)
        return avg_gap, top_points, bottom_points, gaps

    return 0, [], [], []


def visualize_multiple_rois(image_path, pixels1, pixels2, output_path, roi1, roi2):
    """두 개의 ROI를 가진 이미지 시각화"""
    img = cv2.imread(image_path)

    # ROI 테두리 표시
    x1, y1, w1, h1 = roi1
    x2, y2, w2, h2 = roi2
    cv2.rectangle(img, (x1, y1), (x1 + w1, y1 + h1), (255, 255, 255), 2)
    cv2.rectangle(img, (x2, y2), (x2 + w2, y2 + h2), (255, 255, 255), 1)

    # ROI1 처리
    if pixels1:
        height1 = calculate_height(pixels1)
        y_coords1 = [p['position'][1] for p in pixels1]
        min_y1, max_y1 = min(y_coords1), max(y_coords1)
        left_x1 = min(p['position'][0] for p in pixels1)
        cv2.circle(img, (left_x1, min_y1), 5, (0, 0, 255), -1)
        cv2.circle(img, (left_x1, max_y1), 5, (0, 0, 255), -1)
        cv2.putText(img, f"Height 1: {height1}px", 
                   (left_x1 + 10, (min_y1 + max_y1) // 2),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # ROI2 gap 처리
    if pixels2:
        avg_gap, top_points, bottom_points, gaps = calculate_inner_border_height(pixels2)
        if avg_gap > 0 and top_points and bottom_points:
            # 모든 gap 포인트 표시
            min_gap_x = None
            min_gap_value = float('inf')
            min_gap = float('inf')  # min_gap 초기화 추가
    
            # 기존의 모든 gap 라인을 그리는 대신 최소 gap 위치만 표시
            for (x, top_y), (_, bottom_y) in zip(top_points, bottom_points):
                gap = bottom_y - top_y
                # 최소 gap 찾기
                if gap < min_gap_value:
                    min_gap_value = gap
                    min_gap_x = x
                    min_gap = gap  # min_gap 업데이트
    
            # gap 선 표시 부분을 수정

        # 기존의 모든 gap 라인을 그리는 대신 최소 gap 위치만 표시
        for (x, top_y), (_, bottom_y) in zip(top_points, bottom_points):
            gap = bottom_y - top_y
            # 최소 gap 찾기
            if gap < min_gap_value:
                min_gap_value = gap
                min_gap_x = x
        
            # 위쪽 라인의 최하단 점 찾기
            top_cluster = []
            bottom_cluster = []
            for pixel in pixels2:
                px, py = pixel['position']
                if abs(px - x) < 1:  # x 좌표가 같은 픽셀들 그룹화
                    if abs(py - top_y) < 5:  # 위쪽 클러스터
                        top_cluster.append(py)
                    elif abs(py - bottom_y) < 5:  # 아래쪽 클러스터
                        bottom_cluster.append(py)
        
            if top_cluster:
                top_max_y = max(top_cluster)  # 위쪽 라인의 최하단
                top_min_y = min(top_cluster)  # 위쪽 라인의 최상단
            if bottom_cluster:
                bottom_max_y = max(bottom_cluster)  # 아래쪽 라인의 최하단
                bottom_min_y = min(bottom_cluster)  # 아래쪽 라인의 최상단
        
            # 위쪽 라인 표시 (최상단-녹색, 최하단-파란색)
            cv2.circle(img, (int(x), int(top_max_y)), 2, (255, 0, 0), -1)  # 최하단
        
            # 아래쪽 라인 표시 (최상단-녹색)
            cv2.circle(img, (int(x), int(bottom_min_y)), 2, (0, 255, 0), -1)  # 최상단
        
            # 일반적인 gap 선은 더 이상 그리지 않음
            # 최소 gap 위치에서만 선을 그리도록 함
        
        # 최소 gap 위치에 노란색 선 표시
        if min_gap_x is not None:
            for x, y_coords in zip(top_points, bottom_points):
                if x[0] == min_gap_x:
                    top_cluster = []
                    bottom_cluster = []
                    for pixel in pixels2:
                        px, py = pixel['position']
                        if abs(px - min_gap_x) < 1:
                            if abs(py - x[1]) < 5:
                                top_cluster.append(py)
                            elif abs(py - y_coords[1]) < 5:
                                bottom_cluster.append(py)
        
                    top_max_y = max(top_cluster) if top_cluster else x[1]
                    bottom_min_y = min(bottom_cluster) if bottom_cluster else y_coords[1]
        
                    # 최소 gap 위치에 노란색으로 표시
                    cv2.line(img, (int(min_gap_x), int(top_max_y)), 
                            (int(min_gap_x), int(bottom_min_y)), (0, 255, 255), 2)
                    cv2.putText(img, f"Min Gap: {min_gap:.1f}px", 
                                (int(min_gap_x) + 5, int((top_max_y + bottom_min_y) // 2)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                    break



            # 평균 gap 텍스트 표시
            cv2.putText(img, f"Top Gap: {avg_gap:.1f}px", 
                       (roi2[0] + 10, roi2[1] - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
            if gaps:
                # 최대/최소 gap 표시
                max_gap = max(gaps)
                min_gap = min(gaps)
                cv2.putText(img, f"Max: {max_gap:.1f}px, Min: {min_gap:.1f}px", 
                           (roi2[0] + 10, roi2[1] - 40),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
                # 최소 gap 위치에 특별 표시
                if min_gap_x is not None:
                    for x, y_coords in zip(top_points, bottom_points):
                        if x[0] == min_gap_x:
                            # 해당 x 좌표에서의 클러스터 재계산
                            top_cluster = []
                            bottom_cluster = []
                            for pixel in pixels2:
                                px, py = pixel['position']
                                if abs(px - min_gap_x) < 1:
                                    if abs(py - x[1]) < 5:
                                        top_cluster.append(py)
                                    elif abs(py - y_coords[1]) < 5:
                                        bottom_cluster.append(py)
    
                            top_max_y = max(top_cluster) if top_cluster else x[1]
                            bottom_min_y = min(bottom_cluster) if bottom_cluster else y_coords[1]
    
                            # 최소 gap 위치에 빨간색으로 표시
                            cv2.line(img, (int(min_gap_x), int(top_max_y)), 
                                   (int(min_gap_x), int(bottom_min_y)), (0, 0, 255), 2)
                            cv2.putText(img, f"Min Gap: {min_gap:.1f}px", 
                                      (int(min_gap_x) + 5, int((top_max_y + bottom_min_y) // 2)),
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                            break

    cv2.imwrite(output_path, img)
    
def enhance_contrast(image, method='clahe', params=None):
    """
    이미지의 대비를 강화하는 함수

    Parameters:
    image: 입력 이미지 (파일 경로 또는 이미지 배열)
    method: 대비 강화 방법 ('clahe', 'histeq', 'gamma' 중 선택)
    params: 각 방법에 대한 추가 매개변수 딕셔너리

    Returns:
    enhanced_image: 대비가 강화된 이미지
    """
    # 이미지가 경로로 주어진 경우 로드
    if isinstance(image, str):
        img = cv2.imread(image)
    else:
        img = image.copy()

    if params is None:
        params = {}

    if method == 'clahe':
        # CLAHE (Contrast Limited Adaptive Histogram Equalization) 적용
        clip_limit = params.get('clip_limit', 2.0)
        tile_grid_size = params.get('tile_grid_size', (8, 8))

        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)

        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
        cl = clahe.apply(l)

        enhanced = cv2.merge((cl, a, b))
        enhanced = cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)

    elif method == 'histeq':
        # 일반 히스토그램 평활화
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)

        # L 채널에 대해 히스토그램 평활화 적용
        cl = cv2.equalizeHist(l)

        enhanced = cv2.merge((cl, a, b))
        enhanced = cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)

    elif method == 'gamma':
        # 감마 보정
        gamma = params.get('gamma', 1.5)
        lookup_table = np.array([((i / 255.0) ** gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        enhanced = cv2.LUT(img, lookup_table)

    else:
        enhanced = img

    return enhanced

def process_folder(input_folder, output_folder, use_zerodce=False, contrast_method='clahe', contrast_params=None):
    """
    폴더 내 모든 이미지 처리

    Parameters:
    input_folder: 입력 이미지 폴더 경로
    output_folder: 출력 이미지 폴더 경로
    use_zerodce: ZeroDCE 적용 여부 (기본값: False)
    contrast_method: 대비 강화 방법 ('clahe', 'histeq', 'gamma' 중 선택)
    contrast_params: 대비 강화 매개변수
    """
    os.makedirs(output_folder, exist_ok=True)
    results = {}

    image_files = [f for f in os.listdir(input_folder) 
                  if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    image_files.sort()

    for image_file in image_files:
        image_path = os.path.join(input_folder, image_file)

        try:
            if use_zerodce:
                # ZeroDCE 적용
                enhanced_img = zeroDCE(img_path=image_path).numpy()
            else:
                # 원본 이미지 로드
                enhanced_img = cv2.imread(image_path)

            # 대비 강화 적용
            enhanced_img = enhance_contrast(enhanced_img, method=contrast_method, params=contrast_params)

            # 임시로 향상된 이미지 저장
            temp_enhanced_path = os.path.join(output_folder, f"temp_enhanced_{image_file}")
            cv2.imwrite(temp_enhanced_path, enhanced_img)
            processing_path = temp_enhanced_path


            ''' ROI '''
            ''' ROI '''
            ''' ROI '''
            ''' ROI '''

            # ROI 설정
            roi1 = (200, 420, 150, 200)  # (x, y, width, height)
            roi2 = (450, 430, 400, 15)   # (x, y, width, height)
            
            ''' ROI '''
            ''' ROI '''
            ''' ROI '''
            ''' ROI '''

            # 픽셀 찾기
            bright_pixels1 = find_bright_pixels(processing_path, brightness_threshold=190, roi=roi1)
            bright_pixels2 = find_bright_pixels(processing_path, brightness_threshold=190, roi=roi2)

            # 결과 저장
            # results 저장 부분 수정
            if bright_pixels1 or bright_pixels2:
                height1 = calculate_height(bright_pixels1) if bright_pixels1 else 0
            
                # ROI2 계산
                if bright_pixels2:
                    avg_height, _, _, gaps = calculate_inner_border_height(bright_pixels2)
                    min_gap = min(gaps) if gaps else 0
                else:
                    avg_height = 0
                    min_gap = 0
            
                results[image_file] = {
                    'roi1': {
                        'pixels': bright_pixels1,
                        'height': height1
                    },
                    'roi2': {
                        'pixels': bright_pixels2,
                        'inner_height': avg_height,
                        'min_gap': min_gap  # min_gap 추가
                    }
                }
      

            # 결과 이미지 저장
            output_path = os.path.join(output_folder, f"processed_{image_file}")
            visualize_multiple_rois(processing_path, bright_pixels1, bright_pixels2, 
                                  output_path, roi1, roi2)

            # 임시 파일 삭제
            if os.path.exists(temp_enhanced_path):
                os.remove(temp_enhanced_path)

        except Exception as e:
            print(f"Error processing {image_file}: {str(e)}")
            continue


    # 결과 데이터 확인
    print("\nResults dictionary contents:")
    for image_file, data in results.items():
        # ROI2의 최소 gap 계산
        roi2_pixels = data['roi2']['pixels']
        if roi2_pixels:
            _, _, _, gaps = calculate_inner_border_height(roi2_pixels)
            min_gap = min(gaps) if gaps else 0
        else:
            min_gap = 0

        # print(f"\n{image_file}:")
        # print(f"ROI1 height: {data['roi1']['height']}")
        # print(f"ROI2 inner height: {data['roi2']['inner_height']}")
        # print(f"ROI2 min gap: {min_gap:.1f}")
    
    # 통계 분석 및 결과 출력
    roi1_heights = [data['roi1']['height'] for data in results.values()]
    roi2_inner_heights = [data['roi2']['inner_height'] for data in results.values() 
                         if data['roi2']['inner_height'] > 0]

    if roi1_heights:
        print("\n=== ROI1 높이 통계 분석 ===")
        avg_height = mean(roi1_heights)
        std_dev = stdev(roi1_heights) if len(roi1_heights) > 1 else 0
        min_height = min(roi1_heights)
        max_height = max(roi1_heights)

        print(f"평균 높이: {avg_height:.2f} 픽셀")
        print(f"표준편차: {std_dev:.2f} 픽셀")
        print(f"최소 높이: {min_height} 픽셀")
        print(f"최대 높이: {max_height} 픽셀")
        print(f"높이 범위: {min_height} ~ {max_height} 픽셀")
        print(f"오차 범위(±2σ): {avg_height - 2*std_dev:.2f} ~ {avg_height + 2*std_dev:.2f} 픽셀")


    # 위치 분석 수행
    positions, position_stats = analyze_pixel_positions(results)

    
    # Gap 분석 부분 수정
    if roi2_inner_heights:
        print("\n=== ROI2 Gap 분석 ===")
        gaps = [d['roi2']['inner_height'] for d in results.values()]
        min_gaps = [d['roi2']['min_gap'] for d in results.values()]  # 최소 gap 리스트
    
        print(f"평균 Gap: {mean(gaps):.2f} 픽셀")
        print(f"최대 Gap: {max(gaps):.2f} 픽셀")
        print(f"최소 Gap: {min(gaps):.2f} 픽셀")
        print(f"Gap 표준편차: {stdev(gaps):.2f} 픽셀")
        print(f"\n최소 Gap 평균: {mean(min_gaps):.2f} 픽셀")
        print(f"최소 Gap 표준편차: {stdev(min_gaps):.2f} 픽셀")
        print(f"최소 Gap 범위: {min(min_gaps):.2f} ~ {max(min_gaps):.2f} 픽셀")

    # ROI1 높이를 기준으로 실제 거리 변환 (3cm 기준)
    ACTUAL_HEIGHT_CM = 3.0 # ROI1의 실제 높이 (3cm)
    
    print("\n=== 규격 벗어난 이미지 분석 ===")
    # 평균 ROI1 높이 계산
    avg_roi1_height = mean([data['roi1']['height'] for data in results.values() if data['roi1']['height'] > 0])
    
    # 허용 범위를 픽셀로 변환
    pixels_per_cm = avg_roi1_height / ACTUAL_HEIGHT_CM
    min_allowed_pixels = 0.05 * pixels_per_cm  # 0.05cm를 픽셀로 변환
    max_allowed_pixels = 0.15 * pixels_per_cm  # 0.15cm를 픽셀로 변환
    
    print(f"허용 범위: 0.05cm ~ 0.15cm")
    print(f"픽셀 허용 범위: {min_allowed_pixels:.1f}px ~ {max_allowed_pixels:.1f}px")
    
    out_of_spec_images = []
    valid_images = 0  # 유효한 이미지 카운트
    total_processed = 0  # 처리된 총 이미지 카운트
    
    for image_file, data in results.items():
        roi1_height = data['roi1']['height']
        if roi1_height == 0:  # ROI1 높이가 0인 경우 처리
            continue

        total_processed += 1  # 처리된 이미지 카운트 증가

        # 픽셀당 실제 거리(cm) 계산
        cm_per_pixel = ACTUAL_HEIGHT_CM / roi1_height

        # 최소 gap의 실제 거리(cm) 계산
        min_gap_cm = data['roi2']['min_gap'] * cm_per_pixel

        # 허용 범위를 벗어나는 경우 기록
        if min_gap_cm < 0.05 or min_gap_cm > 0.15:
            out_of_spec_images.append({
                'image': image_file,
                'min_gap_cm': min_gap_cm,
                'roi1_height': roi1_height,
                'min_gap_pixels': data['roi2']['min_gap']
            })

    # 정확도 계산
    accuracy = ((total_processed - len(out_of_spec_images)) / total_processed * 100) if total_processed > 0 else 0

    # 종합 결과 출력
    print("\n=== 검사 종합 결과 ===")
    print(f"총 처리된 이미지: {total_processed}개")
    print(f"규격 외 이미지: {len(out_of_spec_images)}개")
    print(f"정확도: {accuracy:.2f}%")
    
    # 규격 벗어난 이미지들 출력
    if out_of_spec_images:
        print("\n=== 부품 높이 3cm 기준 ===")
        print("\n규격을 벗어난 이미지들:")
        for item in out_of_spec_images:
            print(f"\n이미지: {item['image']}")
            print(f"최소 Gap: {item['min_gap_cm']:.3f}cm")
            print(f"ROI1 높이: {item['roi1_height']}픽셀")
            print(f"최소 Gap: {item['min_gap_pixels']:.1f}픽셀")
    else:
        print("\n모든 이미지가 규격 내에 있습니다.")
    
    return results

if __name__ == "__main__":
    input_folder = "test"
    output_folder = "output"

    # 설정
    use_zerodce = False
    contrast_method = 'gamma'  
    contrast_params = {
        'gamma': 1.5           # 감마 보정 파라미터
    }

    results = process_folder(
        input_folder=input_folder,
        output_folder=output_folder,
        use_zerodce=use_zerodce,
        contrast_method=contrast_method,
        contrast_params=contrast_params
    )

Error processing 20.bmp: local variable 'min_gap_x' referenced before assignment
Error processing 25.bmp: local variable 'min_gap_x' referenced before assignment

Results dictionary contents:

=== ROI1 높이 통계 분석 ===
평균 높이: 187.00 픽셀
표준편차: 3.00 픽셀
최소 높이: 183 픽셀
최대 높이: 190 픽셀
높이 범위: 183 ~ 190 픽셀
오차 범위(±2σ): 181.00 ~ 193.00 픽셀

=== ROI2 Gap 분석 ===
평균 Gap: 4.57 픽셀
최대 Gap: 9.52 픽셀
최소 Gap: 0.00 픽셀
Gap 표준편차: 5.28 픽셀

최소 Gap 평균: 3.00 픽셀
최소 Gap 표준편차: 4.24 픽셀
최소 Gap 범위: 0.00 ~ 8.00 픽셀

=== 규격 벗어난 이미지 분석 ===
허용 범위: 0.05cm ~ 0.15cm
픽셀 허용 범위: 3.1px ~ 9.3px

=== 검사 종합 결과 ===
총 처리된 이미지: 4개
규격 외 이미지: 2개
정확도: 50.00%

=== 부품 높이 3cm 기준 ===

규격을 벗어난 이미지들:

이미지: 20.bmp
최소 Gap: 0.000cm
ROI1 높이: 183픽셀
최소 Gap: 0.0픽셀

이미지: 25.bmp
최소 Gap: 0.000cm
ROI1 높이: 188픽셀
최소 Gap: 0.0픽셀


In [ ]:
기존 결과

=== ROI1 높이 통계 분석 ===
평균 높이: 202.00 픽셀
표준편차: 0.00 픽셀
최소 높이: 202 픽셀
최대 높이: 205 픽셀
높이 범위: 202 ~ 205 픽셀
오차 범위(±2σ): 202.00 ~ 202.00 픽셀

=== ROI1 픽셀 위치 일관성 분석 ===
중심점 X 좌표 평균: 241.76
중심점 X 좌표 표준편차: 1.02
중심점 Y 좌표 평균: 300.63
중심점 Y 좌표 표준편차: 0.83

In [ ]:
Results dictionary contents:

=== ROI1 높이 통계 분석 ===
평균 높이: 202.00 픽셀
표준편차: 0.00 픽셀
최소 높이: 201 픽셀
최대 높이: 203 픽셀
높이 범위: 201 ~ 203 픽셀
오차 범위(±2σ): 202.00 ~ 202.00 픽셀

=== ROI2 Gap 분석 ===
평균 Gap: 11.45 픽셀
최대 Gap: 12.94 픽셀
최소 Gap: 9.07 픽셀
Gap 표준편차: 1.10 픽셀

최소 Gap 평균: 8.00 픽셀
최소 Gap 표준편차: 1.00 픽셀
최소 Gap 범위: 6.00 ~ 11.00 픽셀

=== 규격 벗어난 이미지 분석 ===
허용 범위: 0.095cm ~ 0.15cm

부품 높이 3cm 기준

규격을 벗어난 이미지들:

이미지: moved_31.jpeg
최소 Gap: 0.163cm
ROI1 높이: 203픽셀
최소 Gap: 11.0픽셀

이미지: moved_32.jpeg
최소 Gap: 0.163cm
ROI1 높이: 203픽셀
최소 Gap: 11.0픽셀

이미지: moved_41.jpeg
최소 Gap: 0.163cm
ROI1 높이: 203픽셀
최소 Gap: 11.0픽셀

이미지: moved_44.jpeg
최소 Gap: 0.089cm
ROI1 높이: 202픽셀
최소 Gap: 6.0픽셀

이미지: moved_60.jpeg
최소 Gap: 0.089cm
ROI1 높이: 202픽셀
최소 Gap: 6.0픽셀
